# Prediciendo con variables categóricas

En este notebook vamos a ver cómo podemos tratar las variables categóricas de un dataset, ya que pueden ser una fuente muy importante de información.

In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [30]:
df = pd.read_csv("data/transactions.csv")

In [31]:
df.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485


Como podemos ver, en este dataset no solo tenemos variables numéricas, sino también categóricas. Estas variables pueden aportar información, por lo que, de no utilizarlas, estaríamos perdiendo información que podría mejorar nuestro modelo.

Como ya vimos en su día en el apartado de Feature Engineering, no podemos pasarle al algoritmo las variables categóricas como tal, exigen cierto tratamiento. Tenemoms que pasarlas a numéricas, pero ¿cómo?

Habíamos visto 2 formas:
  - Mapeo de los diferentes valores a número: se trata de mapear cada posible valor de la variable categórica a un valor numérico. Haciendo esto le estamos asignando cierto orden a uno sobre otro, que puede no ser así o no tan cuantificable. Puede servirnos como primera aproximación o para algún caso concreto, pero por lo general lo evitaremos en favor de get_dummies
  - Variables dummy: consiste en crear tantas variables nuevas como valores tenga la varaible en cuestión (menos 1 valor, como explicamos en su día, por el tema de poder expresar la combinación de 0's o no). De este mdo, las varaibles nuevas solo valdrán 1 o 0, y se corresponderán con el valor de la variable. De este modo se transfiere la información al modelo sin incurrir en el orden o prioridad del método anterior. Recordemos cómo se hacía:

In [32]:
dummy_gender = pd.get_dummies(df["Gender"], prefix = "Gender")
dummy_city_tier = pd.get_dummies(df["City Tier"], prefix = "City")

Vamos a ver qué hemos hecho:

In [34]:
dummy_gender.head()

,Gender_Female,Gender_Male
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [35]:
dummy_city_tier.head()

,City_Tier 1,City_Tier 2,City_Tier 3
0,1,0,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


In [36]:
column_names = df.columns.values.tolist()
column_names

['Transaction ID',
 'Age',
 'Items',
 'Monthly Income',
 'Transaction Time',
 'Record',
 'Gender',
 'City Tier',
 'Total Spend']

Vale, nos hemos creado las columnas pero... ¿dónde están?

Tranquilo, realmente lo que nos hemos creado son DataFrames que están aislados, no los hemos unido a nada. Las columnas aún no están en el DataFrame principal, así que las uniremos.

Empezamos por **Gender**:

In [37]:
df_new = df[column_names].join(dummy_gender)
column_names = df_new.columns.values.tolist()
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0


Y pasamos a **City:**

In [38]:
df_new = df_new[column_names].join(dummy_city_tier)
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0,1,0,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0,0,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1,0,1,0
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0,1,0,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0,0,1,0


Llegados a este punto, podremos crearnos nuestro modelo de regresión lineal con las varaibles que queramos. Para hacer este ejemplo, nos basaremos en las variables de "Monthly Income", "Transaction Time", "Record", y aquellas que acabamos de obtener a partir de "Gender" y "City". Sin embargo, para hacerlo bien ya sabemos lo que deberíamos hacer.

In [126]:
feature_cols = ["Monthly Income", "Transaction Time", 
                "Gender_Female", "Gender_Male", 
                "City_Tier 1", "City_Tier 2", "City_Tier 3",
                "Record"]

In [127]:
X = df_new[feature_cols]
Y = df_new["Total Spend"]

In [128]:
lm = LinearRegression()
lm.fit(X,Y)

LinearRegression()

In [129]:
print(lm.intercept_)
print(lm.coef_)

-79.4171303013718
[ 1.47538980e-01  1.54946125e-01 -1.31025013e+02  1.31025013e+02
  7.67643260e+01  5.51389743e+01 -1.31903300e+02  7.72233446e+02]


In [130]:
list(zip(feature_cols, lm.coef_))

[('Monthly Income', 0.14753898049205738),
 ('Transaction Time', 0.1549461254958957),
 ('Gender_Female', -131.02501325554607),
 ('Gender_Male', 131.0250132555463),
 ('City_Tier 1', 76.76432601049517),
 ('City_Tier 2', 55.13897430923261),
 ('City_Tier 3', -131.90330031972786),
 ('Record', 772.2334457445645)]

In [131]:
lm.score(X,Y)

0.9179923586131016

Como puedes observar, el modelo devuelve un resultado bastante bueno, aunque probablemente estemos haceindo overfitting...


El modelo puede ser escrito como:


**Total_Spend** = -79.41713030137362 + **'Monthly Income'** * 0.14753898049205738 + **'Transaction Time'** * 0.15494612549589545+**'Gender_Female'** * -131.02501325554567 + **'Gender_Male'** * 131.0250132555456+**'City_Tier 1'** * 76.76432601049527 + **'City_Tier 2'** * 55.138974309232474 + **'City_Tier 3'** * -131.9033003197278 + **'Record'** * 772.2334457445648

  * Si es hombre y vive en CT1:
    - **Total_Spend** = 128.37220896466724 + **'Monthly Income'** * 0.14753898049205738 + **'Transaction Time'** * 0.15494612549589545+ **'Record'** * 772.2334457445648
  * Si es hombre y vive en CT2:
    - **Total_Spend** = 106.74685726340445 + **'Monthly Income'** * 0.14753898049205738 + **'Transaction Time'** * 0.15494612549589545 + **'Record'** * 772.2334457445648
  * Si es hombre y vive en CT3:
    - **Total_Spend** = -80.29541736555583 + **'Monthly Income'** * 0.14753898049205738 + **'Transaction Time'** * 0.15494612549589545+ **'Record'** * 772.2334457445648
  * Si es mujer y vive en CT1:
    - **Total_Spend** = -79.41713030137362 + **'Monthly Income'** * 0.14753898049205738 + **'Transaction Time'** * 0.15494612549589545 - 131.0250132555456+ 76.76432601049527 + **'Record'** * 772.2334457445648
  * Si es mujer y vive en CT2:
    - **Total_Spend** = -79.41713030137362 + **'Monthly Income'** * 0.14753898049205738 + **'Transaction Time'** * 0.15494612549589545 - 131.0250132555456+ 55.138974309232474  + **'Record'** * 772.2334457445648
  * Si es mujer y vive en CT3:
    - **Total_Spend** = -79.41713030137362 + **'Monthly Income'** * 0.14753898049205738 + **'Transaction Time'** * 0.15494612549589545 - 131.0250132555456-131.9033003197278 + **'Record'** * 772.2334457445648
    
Si queremos añadir las predicciones sobre un dataframe, podemos utilizar la fórmula o ir directamente a las predicciones del modelo:

In [132]:
df_new["prediction"] = lm.predict(X)

In [133]:
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3,prediction
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0,1,0,0,4903.696720
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0,0,1,0,4799.434826
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1,0,1,0,5157.082504
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0,1,0,0,8068.012996
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0,0,1,0,3581.980335


Y para completar el estudio, analizamos la varianza que no explicamos del modelo:

In [134]:
# Suma de los cuadrados de las diferencias:
SSD = np.sum((df_new["prediction"] - df_new["Total Spend"])**2)
SSD

1517733985.340816

In [135]:
# Desviación típica de los residuos:
RSE = np.sqrt(SSD/(len(df_new)-len(feature_cols)-1))
RSE

803.1318809818165

In [136]:
sales_mean=np.mean(df_new["Total Spend"])
sales_mean

In [141]:
# Para obtener el % de varianza no explicada:
error = RSE/sales_mean
error*100

13.03113568029416

### EJERCICIO:

1. Prueba el modelo con un porcentaje de datos de test del 30%

2. Realiza un modelo solo con las variables numéricas y comprueba el resultado.

3. Repite lo mismo solo con las categóricas que te has creado.

4. Repite ahora añadiendo tanto las categóricas como las numéricas y realiza los pasos que creas oportunos para eliminarte las variables que no aporten, y calcula el resultado para compararlo con los otros 2

## Eliminar variables dummy redundantes

En su día dijimos que no aportaba nada crear una variable para todos y cada uno de los valores de la varaible categórica. Si eliminamos una de esas k variables obtendremos el mismo valor, veámoslo con un ejemplo:

Si traduces a un caso concreto lo que tenemos, sería algo con demasiados grados de libertad, algo como:
$10 = a + b*4 + c*2$

En este caso, existen varias combinaciones. En cambio, si utilizamos la reducción que vimos (que solamente era quitar una de las diferentes variables), eliminamos ese grado de libertad y lo dejamos más fijo, reduciendo posibles interacciones que no aportaran nada al modelo

In [142]:
dummy_gender = pd.get_dummies(df["Gender"], prefix="Gender").iloc[:,1:]
dummy_gender.head()

,Gender_Male
0,0
1,0
2,1
3,0
4,0


In [143]:
dummy_city_tier = pd.get_dummies(df["City Tier"], prefix="City").iloc[:,1:]
dummy_city_tier.head()

,City_Tier 2,City_Tier 3
0,0,0
1,1,0
2,1,0
3,0,0
4,1,0


Volvemos a hacer el join, pero habiendo eliminado una columna de cada conversión:

In [144]:
column_names = df.columns.values.tolist()
df_new = df[column_names].join(dummy_gender)
column_names = df_new.columns.values.tolist()
df_new = df_new[column_names].join(dummy_city_tier)
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Male,City_Tier 2,City_Tier 3
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,0,0,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,0,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,1,1,0
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,0,0,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,0,1,0


In [145]:
feature_cols = ["Monthly Income", "Transaction Time", "Gender_Male", "City_Tier 2", "City_Tier 3", "Record"]
X = df_new[feature_cols]
Y = df_new["Total Spend"]
lm = LinearRegression()
lm.fit(X,Y)

LinearRegression()

In [146]:
print(lm.intercept_)

-133.67781754642238


In [147]:
list(zip(feature_cols, lm.coef_))

[('Monthly Income', 0.14753898049205744),
 ('Transaction Time', 0.15494612549589704),
 ('Gender_Male', 262.05002651109584),
 ('City_Tier 2', -21.625351701262808),
 ('City_Tier 3', -208.6676263302231),
 ('Record', 772.2334457445636)]

In [148]:
lm.score(X,Y)

0.9179923586131016

Como puedes comprobar, la métrica sigeu siendo igual, pues lo único que hemos hecho es quitarle columnas que no paortaban información.,

#### Coeficientes con todas las variables en el modelo:
* 'Monthly Income': 0.14753898049205738,
* 'Transaction Time': 0.15494612549589545,
* 'Gender_Female': -131.02501325554567,
* 'Gender_Male': 131.0250132555456,
* 'City_Tier 1': 76.76432601049527,
* 'City_Tier 2': 55.138974309232474,
* 'City_Tier 3': -131.9033003197278,
* 'Record': 772.2334457445648
 
 
#### Coeficientes tras enmascarar las variables dummy pertinentes
* 'Monthly Income': 0.14753898049205744,
* 'Transaction Time': 0.15494612549589631,
* 'Gender_Male': 262.05002651109595,
* 'City_Tier 2': -21.62535170126296,
* 'City_Tier 3': -208.66762633022324,
* 'Record': 772.2334457445635


#### ¿Qué ha cambiado?

  - **Gender**:
    - Gender_Male: 131.02 -> 262.05 (+131.02)
    - Gender_Female: 131.02 -> 0 (-131.02)
    
  - **CT**:
    - CT1: 76.76 -> 0 (-76.76)
    - CT1: 55.13 -> -21.62 (-76.76)
    - CT1: -208.66 -> 0 (-76.76)
    

### EJERCICIO:

1. Utiliza lo que has aprendido para realizar un modelo lineal sobre el dataset del Titanic, donde la separación train-test sea de 0.3

2. ¿Realmente aportan información útil las variables categóricas de ese ejemplo? Responde a la pregunta entrenando el modelo sin categóricas, con todas y solo con categóricas, manteniendo el porcentaj de juego:

In [155]:
df = pd.read_csv("../../../data/Titanic.csv", sep='\t')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
151,152,1,1,"Pears, Mrs. Thomas (Edith Wearne)",female,22.0,1,0,113776,66.6000,C2,S
152,153,0,3,"Meo, Mr. Alfonzo",male,55.5,0,0,A.5. 11206,8.0500,NaN,S
153,154,0,3,"van Billiard, Mr. Austin Blyler",male,40.5,0,2,A/5. 851,14.5000,NaN,S
154,155,0,3,"Olsen, Mr. Ole Martin",male,NaN,0,0,Fa 265302,7.3125,NaN,S
